In [833]:
# !pip install catboost

# **Библиотеки**

In [834]:
import numpy as np
import pandas as pd
import sklearn
import catboost
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# **Data**

In [835]:
data = pd.read_csv('/content/train.csv')
data

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,3,30.40,82.15,Male,0,No,No,No,No phone service,DSL,No,No,No,Yes,No,No,Month-to-month,No,Electronic check,0
5278,50,44.45,2188.45,Male,0,Yes,No,No,No phone service,DSL,Yes,No,No,Yes,Yes,No,One year,Yes,Bank transfer (automatic),0
5279,1,55.05,55.05,Male,0,No,No,Yes,No,DSL,No,No,Yes,Yes,No,No,Month-to-month,No,Mailed check,0
5280,29,76.00,2215.25,Female,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),0


# **Data processing**

In [836]:
data.isna().sum()

ClientPeriod                0
MonthlySpending             0
TotalSpent                  0
Sex                         0
IsSeniorCitizen             0
HasPartner                  0
HasChild                    0
HasPhoneService             0
HasMultiplePhoneNumbers     0
HasInternetService          0
HasOnlineSecurityService    0
HasOnlineBackup             0
HasDeviceProtection         0
HasTechSupportAccess        0
HasOnlineTV                 0
HasMovieSubscription        0
HasContractPhone            0
IsBillingPaperless          0
PaymentMethod               0
Churn                       0
dtype: int64

### **NaN значения в данных отсутствуют**

In [837]:
data.describe()

,ClientPeriod,MonthlySpending,IsSeniorCitizen,Churn
count,5282.000000,5282.000000,5282.000000,5282.000000
mean,32.397009,64.924754,0.159409,0.262022
std,24.550326,30.176464,0.366092,0.439776
min,0.000000,18.250000,0.000000,0.000000
25%,9.000000,35.462500,0.000000,0.000000
50%,29.000000,70.400000,0.000000,0.000000
75%,55.000000,90.050000,0.000000,1.000000
max,72.000000,118.750000,1.000000,1.000000


In [838]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientPeriod              5282 non-null   int64  
 1   MonthlySpending           5282 non-null   float64
 2   TotalSpent                5282 non-null   object 
 3   Sex                       5282 non-null   object 
 4   IsSeniorCitizen           5282 non-null   int64  
 5   HasPartner                5282 non-null   object 
 6   HasChild                  5282 non-null   object 
 7   HasPhoneService           5282 non-null   object 
 8   HasMultiplePhoneNumbers   5282 non-null   object 
 9   HasInternetService        5282 non-null   object 
 10  HasOnlineSecurityService  5282 non-null   object 
 11  HasOnlineBackup           5282 non-null   object 
 12  HasDeviceProtection       5282 non-null   object 
 13  HasTechSupportAccess      5282 non-null   object 
 14  HasOnlin

### **Заметим, что у признака TotalSpent тип - object, но этот признак отвечает за всю потраченную сумму, соответсвенно его тип должен быть float, попробуем найти неправильные данные в этой колонке**

In [839]:
temp = data.sort_values(by = 'TotalSpent')
temp.head(11)

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
4598,0,56.05,,Female,0,Yes,Yes,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),0
2543,0,19.70,,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,0
4551,0,52.55,,Female,0,Yes,Yes,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),0
1707,0,73.35,,Female,0,Yes,Yes,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,0
3078,0,80.85,,Female,0,Yes,Yes,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,0
4326,0,25.35,,Male,0,Yes,Yes,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,0
1048,0,25.75,,Male,0,Yes,Yes,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,0
3697,0,20.00,,Female,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,0
4002,0,61.90,,Male,0,No,Yes,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),0
3343,5,24.30,100.2,Male,0,No,No,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,No,Mailed check,0


### **Первые 9 элементов, это пробелы, попробуем их восстановить исходя из логики: первые два столбца это клиентский период, а второй - ежемесячные расходы. Стало быть их произведение это и будет наш столбец TotalSpent, также заметим что при этих индексах ClientPeriod должен равнятся на нулю, а единице, так как ежемесячная трата был**

In [840]:
data.loc[4598,'TotalSpent'] = float(56.05)
data.loc[4002,'TotalSpent'] = float(61.90)
data.loc[3697,'TotalSpent'] = float(20.00)
data.loc[4326,'TotalSpent'] = float(25.35)
data.loc[3078,'TotalSpent'] = float(80.85)
data.loc[1707,'TotalSpent'] = float(73.35)
data.loc[4551,'TotalSpent'] = float(52.55)
data.loc[2543,'TotalSpent'] = float(19.70)
data.loc[1048,'TotalSpent'] = float(25.75)

data.loc[4598,'ClientPeriod'] = int(1)
data.loc[4002,'ClientPeriod'] = int(1)
data.loc[3697,'ClientPeriod'] = int(1)
data.loc[4326,'ClientPeriod'] = int(1)
data.loc[3078,'ClientPeriod'] = int(1)
data.loc[1707,'ClientPeriod'] = int(1)
data.loc[4551,'ClientPeriod'] = int(1)
data.loc[2543,'ClientPeriod'] = int(1)
data.loc[1048,'ClientPeriod'] = int(1)

data["TotalSpent"] = data.TotalSpent.astype(float)

In [841]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientPeriod              5282 non-null   int64  
 1   MonthlySpending           5282 non-null   float64
 2   TotalSpent                5282 non-null   float64
 3   Sex                       5282 non-null   object 
 4   IsSeniorCitizen           5282 non-null   int64  
 5   HasPartner                5282 non-null   object 
 6   HasChild                  5282 non-null   object 
 7   HasPhoneService           5282 non-null   object 
 8   HasMultiplePhoneNumbers   5282 non-null   object 
 9   HasInternetService        5282 non-null   object 
 10  HasOnlineSecurityService  5282 non-null   object 
 11  HasOnlineBackup           5282 non-null   object 
 12  HasDeviceProtection       5282 non-null   object 
 13  HasTechSupportAccess      5282 non-null   object 
 14  HasOnlin

# **Разделим данные**

In [842]:
from sklearn.model_selection import train_test_split

y = data['Churn']

X = data.drop(columns = 'Churn')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.81, random_state=42, stratify=y)

**Так как в датасете есть категориальные фичи, закодируем их с помощью catboost encoder**

In [844]:
# !pip install category_encoders

In [845]:
from category_encoders.cat_boost import CatBoostEncoder

cbe = CatBoostEncoder(verbose=False)
X_train = cbe.fit_transform(X_train, y_train)
X_train

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
4443,57,97.55,5598.00,0.262038,0,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038
3232,41,19.75,806.95,0.262038,0,0.262038,0.262038,0.131019,0.131019,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.262038,0.131019,0.131019,0.262038
1889,60,60.80,3603.45,0.131019,0,0.131019,0.131019,0.262038,0.262038,0.262038,0.131019,0.131019,0.131019,0.262038,0.131019,0.131019,0.262038,0.262038,0.131019
2706,62,104.30,6613.65,0.087346,0,0.087346,0.087346,0.087346,0.087346,0.131019,0.262038,0.262038,0.087346,0.131019,0.087346,0.087346,0.087346,0.087346,0.262038
2466,5,40.00,223.45,0.131019,0,0.131019,0.065510,0.131019,0.131019,0.131019,0.131019,0.131019,0.065510,0.087346,0.262038,0.262038,0.262038,0.131019,0.131019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,70,19.55,1462.05,0.269696,1,0.193969,0.306746,0.265555,0.250128,0.072193,0.072193,0.072193,0.072193,0.072193,0.072193,0.072193,0.028548,0.330939,0.143928
1582,8,100.60,819.40,0.254557,1,0.326622,0.306643,0.265486,0.283228,0.406695,0.414643,0.395621,0.220989,0.408056,0.299082,0.283578,0.419316,0.330809,0.443260
3914,64,99.00,6375.80,0.269570,0,0.193876,0.306876,0.265677,0.250006,0.407004,0.137641,0.395938,0.221508,0.408335,0.299500,0.284004,0.028521,0.331073,0.167125
3573,63,83.00,5243.05,0.269444,1,0.326929,0.306773,0.265608,0.283628,0.189016,0.137530,0.205901,0.221360,0.152522,0.299322,0.342369,0.113035,0.161738,0.166947


**Отскалируем данные**

In [846]:
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
X_train = SS.fit_transform(X_train)

X_test = cbe.transform(X_test)
X_test = SS.transform(X_test)

# **Модель Catboost**

In [847]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=505,
    depth = 2,
    random_seed=42,
    loss_function="Logloss",
    custom_loss=['AUC', 'Accuracy'],
    l2_leaf_reg = 3.180242242411711,
    min_data_in_leaf= 2,
    grow_policy = 'Lossguide',
    eta = 0.043562,
    subsample = 0.6983999999999999,
)

## **Тренировка модели**

In [ ]:
model.fit(
    X_train, y_train,
    verbose=False
)

## **Предсказания на тренировочном датасете**

In [849]:
y_pred = model.predict_proba(X_test)
y = y_pred[:, 1]

from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y)

0.8662838728878353

# **Предсказания на тестовом датасете**

In [850]:
X_test = pd.read_csv('/content/test.csv')

X_test.loc[177,'TotalSpent'] = float(19.85)
X_test.loc[767,'TotalSpent'] = float(20.25)

X_test.loc[177,'ClientPeriod'] = int(1)
X_test.loc[767,'ClientPeriod'] = int(1)

X_test["TotalSpent"] = X_test.TotalSpent.astype(float)

X_test = cbe.transform(X_test)
X_test = SS.transform(X_test)

In [851]:
y_pred = model.predict_proba(X_test)
y = y_pred[:, 1]

In [852]:
vec = []

for i in range(0, 1761):
  vec.append(i)


ans = pd.DataFrame({'Id': vec, 'Churn': y})

ans.to_csv('ans.csv', index = False)